In [ ]:
import csv
from pathlib import Path
data_path = Path('..') / 'quick_analysis_results.csv'
rows = list(csv.DictReader(open(data_path)))
speedups = [2*float(r['full_time'])/float(r['quick_time']) for r in rows]
circuits = [f"{int(r['qubits'])*2}q" for r in rows]


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 4))
plt.bar(circuits, speedups)
plt.ylabel('Speedup (×)')
plt.xlabel('Circuit')
plt.title('Relative Speedups')
plt.show()


## Parallel GHZ subsystems

Independent GHZ chains built with ``many_ghz_subsystems`` remain disjoint
throughout the circuit.  The planner therefore tags a single step with
parallel metadata that the scheduler can execute concurrently.  The cell
below compares the cost model's sequential and parallel estimates for
eight groups of six qubits.


In [3]:
from benchmarks.parallel_circuits import many_ghz_subsystems
from quasar.cost import Backend
from quasar.partitioner import Partitioner
from quasar.planner import Planner, _add_cost, _parallel_simulation_cost, _simulation_cost

num_groups, group_size = 8, 6
circuit = many_ghz_subsystems(num_groups=num_groups, group_size=group_size)

planner = Planner()
plan = planner.plan(circuit, backend=Backend.TABLEAU)
step = plan.steps[0]
groups = Partitioner().parallel_groups(circuit.gates)

estimator = planner.estimator
sequential = None
for qubits, gates in groups:
    n = len(qubits)
    num_meas = sum(1 for gate in gates if gate.gate.upper() in {"MEASURE", "RESET"})
    num_1q = sum(
        1
        for gate in gates
        if len(gate.qubits) == 1 and gate.gate.upper() not in {"MEASURE", "RESET"}
    )
    num_2q = sum(
        1
        for gate in gates
        if len(gate.qubits) > 1 and gate.gate.upper() not in {"MEASURE", "RESET"}
    )
    cost = _simulation_cost(estimator, Backend.TABLEAU, n, num_1q, num_2q, num_meas)
    sequential = cost if sequential is None else _add_cost(sequential, cost)

parallel_cost = _parallel_simulation_cost(estimator, Backend.TABLEAU, groups)

print(f"Plan step parallel groups: {step.parallel}")
print(
    f"Sequential estimate: {sequential.time:.6f}s, Parallel estimate: {parallel_cost.time:.6f}s"
)
if parallel_cost.time > 0:
    print(f"Estimated speed-up: {sequential.time / parallel_cost.time:.2f}x")


Plan step parallel groups: ((0, 1, 2, 3, 4, 5), (6, 7, 8, 9, 10, 11), (12, 13, 14, 15, 16, 17), (18, 19, 20, 21, 22, 23), (24, 25, 26, 27, 28, 29), (30, 31, 32, 33, 34, 35), (36, 37, 38, 39, 40, 41), (42, 43, 44, 45, 46, 47))
Sequential estimate: 6912.000000s, Parallel estimate: 864.000000s
Estimated speed-up: 8.00x
